### Exercise 1.1
Detecting and tracking moving cars using frame differencing and background subtraction techniques

In [1]:
import cv2
import numpy as np

Test if the video loads up. Press escape to end video.

In [6]:
# Specify the path to the video file
video_path1 = 'Traffic_Laramie_1.mp4'
video_path2 = 'Traffic_Laramie_2.mp4'
cap = cv2.VideoCapture(video_path1)

while True:
    ret, frame = cap.read()
    
    cv2.imshow("Frame", frame)
    
    key = cv2.waitKey(30)
    if key == 27:
        break

cap.release()
cv2.destroyAllWindows()

Create a background subtractor for the frames in the video, to see any movements in the video.

In [7]:
cap = cv2.VideoCapture(video_path1)

# Object detection from camera
object_detector = cv2.createBackgroundSubtractorMOG2()

while True:
    ret, frame = cap.read()
    
    # Object Detection
    mask = object_detector.apply(frame)
    
    cv2.imshow("Mask", mask)
    
    key = cv2.waitKey(30)
    if key == 27:
        break

cap.release()
cv2.destroyAllWindows()

Highlight the movement detection from the masked video into the original video by drawing contours.

In [9]:
cap = cv2.VideoCapture(video_path1)

# Object detection from camera
object_detector = cv2.createBackgroundSubtractorMOG2()

while True:
    ret, frame = cap.read()
    
    # Object Detection
    mask = object_detector.apply(frame)
    contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    for cnt in contours:
        cv2.drawContours(frame, [cnt], -1, (0, 255, 0), 2)
    
    cv2.imshow("Mask", mask)
    cv2.imshow("Frame", frame)
    
    key = cv2.waitKey(30)
    if key == 27:
        break

cap.release()
cv2.destroyAllWindows()

Draw contours only if the area of the contour area is not above 500.

In [10]:
cap = cv2.VideoCapture(video_path1)

# Object detection from camera
object_detector = cv2.createBackgroundSubtractorMOG2()

while True:
    ret, frame = cap.read()
    
    # Object Detection
    mask = object_detector.apply(frame)
    contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    for cnt in contours:
        # Calculate area and remove small elements
        area = cv2.contourArea(cnt)
        if area > 500:
            cv2.drawContours(frame, [cnt], -1, (0, 255, 0), 2)
    
    cv2.imshow("Mask", mask)
    cv2.imshow("Frame", frame)
    
    key = cv2.waitKey(30)
    if key == 27:
        break

cap.release()
cv2.destroyAllWindows()

Get the width and height to draw the Region Of Interest(ROI) for the main street in the video.

In [12]:
cap = cv2.VideoCapture(video_path1)

# Object detection from camera
object_detector = cv2.createBackgroundSubtractorMOG2()

while True:
    ret, frame = cap.read()
    height, width, _ = frame.shape
    print("height:",height, "width:", width)
    
    # Extract region of interest(ROI)
    roi = frame[270:720, 0: 1039]
    
    # Draw the region of interest (main street) on the frame
    cv2.rectangle(frame, (0, 270), (1039, 590), (0, 0, 255), 2)
    cv2.putText(frame, "Main Street", (0, 265), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
    
    # Object Detection
    mask = object_detector.apply(roi)
    contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    for cnt in contours:
        # Calculate area and remove small elements
        area = cv2.contourArea(cnt)
        if area > 500:
            cv2.drawContours(roi, [cnt], -1, (0, 255, 0), 2)
    
    cv2.imshow("ROI", roi)
    cv2.imshow("Mask", mask)
    cv2.imshow("Frame", frame)
    
    key = cv2.waitKey(30)
    if key == 27:
        break

cap.release()
cv2.destroyAllWindows()

height: 600 width: 1040
height: 600 width: 1040
height: 600 width: 1040
height: 600 width: 1040
height: 600 width: 1040
height: 600 width: 1040
height: 600 width: 1040
height: 600 width: 1040
height: 600 width: 1040
height: 600 width: 1040
height: 600 width: 1040
height: 600 width: 1040
height: 600 width: 1040
height: 600 width: 1040
height: 600 width: 1040
height: 600 width: 1040
height: 600 width: 1040
height: 600 width: 1040
height: 600 width: 1040
height: 600 width: 1040
height: 600 width: 1040
height: 600 width: 1040
height: 600 width: 1040
height: 600 width: 1040
height: 600 width: 1040
height: 600 width: 1040
height: 600 width: 1040
height: 600 width: 1040
height: 600 width: 1040
height: 600 width: 1040
height: 600 width: 1040
height: 600 width: 1040
height: 600 width: 1040
height: 600 width: 1040
height: 600 width: 1040
height: 600 width: 1040


Perform frame differencing by comparing the current grayscale frame (gray2) with the grayscale frame of the previous time step (gray1), to identify changes in the scene. Draw a bounding rectangle for every car but ignore those with width less than 100 and height less than 25 (assuming those entities are not vehicles). Applied gaussian blur to the mask to make the detection better.

In [13]:
cap = cv2.VideoCapture(video_path1)

ret, frame1 = cap.read()
height, width, _ = frame1.shape

# Convert the first frame to grayscale
gray1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)

# Object detection from a stable camera
object_detector = cv2.createBackgroundSubtractorMOG2(history=100)

while True:
    ret, frame2 = cap.read()
    
    if not ret:
        break
    
    height, width, _ = frame2.shape
    
    # Convert the current frame to grayscale
    gray2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
    
    # Extract region of interest (ROI)
    roi = frame2[270:720, 0: 1039]
    
    # Object Detection
    mask = object_detector.apply(roi)
    
    # Apply Gaussian blur to the mask
    blurred_mask = cv2.GaussianBlur(mask, (31, 31), 0)
    
    _, blurred_mask = cv2.threshold(blurred_mask, 50, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(blurred_mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    for cnt in contours:
        # Calculate area and remove small elements
        area = cv2.contourArea(cnt)
        if area > 500:
            x, y, w, h = cv2.boundingRect(cnt)
            # Ignore bounding boxes with width or height less than a certain threshold
            if w > 100 and h > 25:
                cv2.rectangle(roi, (x, y), (x + w, y + h), (0, 255, 0), 1)
    
    # Draw the region of interest (main street) on the frame
    cv2.rectangle(frame2, (0, 270), (1039, 590), (0, 0, 255), 2)
    cv2.putText(frame2, "Main Street", (0, 265), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
    
    cv2.imshow("Frame", frame2)
    
    key = cv2.waitKey(30)
    if key == 27:
        break

cap.release()
cv2.destroyAllWindows()

### Exercise 1.2

Counting the number of cars that go from city's downtown to the city centre for a specific time interval

In [3]:
import cv2
import numpy as np
import time
import pandas as pd

The code below uses two counting lines (line_start_first to line_end_first and line_start_second to line_end_second) to ensure accurate and reliable car counting, where the first line marks entry into the monitored region, and the second line confirms exit, preventing double counting and allowing for flexible region definition.The code also employs center circles within each bounding box to facilitate car motion detection. These circles aid in accurately tracking the movement of cars as they traverse the monitored area, enhancing the precision of the counting mechanism. After running both videos, the total car count and cars per minute are calculated and displayed in a Pandas DataFrame. The DataFrame includes video names, total car count, and cars per minute for each video.

In [14]:
# Definition to process the video and start the car counting using two lines
def process_video(video_path, line_start_first, line_end_first, line_start_second, line_end_second):
    cap = cv2.VideoCapture(video_path)

    ret, frame1 = cap.read()
    height, width, _ = frame1.shape

    object_detector = cv2.createBackgroundSubtractorMOG2(history=100)

    car_count = 0
    crossed_line_first = False
    crossed_line_second = False
    crossed_line_time_first = 0
    crossed_line_time_second = 0
    detection_delay = 2

    while True:
        ret, frame2 = cap.read()

        if not ret:
            break

        gray2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)

        mask = object_detector.apply(frame2)
        blurred_mask = cv2.GaussianBlur(mask, (31, 31), 0)

        _, blurred_mask = cv2.threshold(blurred_mask, 50, 255, cv2.THRESH_BINARY)
        contours, _ = cv2.findContours(blurred_mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

        for cnt in contours:
            area = cv2.contourArea(cnt)
            if area > 500:
                x, y, w, h = cv2.boundingRect(cnt)
                if w > 100 and h > 25:
                    cv2.rectangle(frame2, (x, y), (x + w, y + h), (0, 255, 0), 1)

                    center_x, center_y = x + w // 2, y + h // 2
                    cv2.circle(frame2, (center_x, center_y), 5, (255, 0, 0), -1)

                    if line_end_first[1] < center_y < line_start_first[1] and line_start_first[0] < center_x < line_end_first[0]:
                        current_time_first = time.time()
                        if not crossed_line_first and (current_time_first - crossed_line_time_first) > detection_delay:
                            crossed_line_first = True
                            crossed_line_time_first = current_time_first
                            crossed_line_second = False

                    if line_end_second[1] < center_y < line_start_second[1] and line_start_second[0] < center_x < line_end_second[0]:
                        current_time_second = time.time()
                        if not crossed_line_second and crossed_line_first and (current_time_second - crossed_line_time_second) > detection_delay:
                            car_count += 1
                            crossed_line_second = True
                            crossed_line_time_second = current_time_second
                        else:
                            crossed_line_second = False

        cv2.line(frame2, line_start_first, line_end_first, (0, 0, 255), 2)
        cv2.line(frame2, line_start_second, line_end_second, (255, 255, 0), 2)

        cv2.putText(frame2, f"Car Count: {car_count}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        cv2.imshow("Frame", frame2)

        key = cv2.waitKey(30)
        if key == 27:
            break

    cap.release()
    cv2.destroyAllWindows()

    return car_count

# Define line parameters for both videos
line_start_first = (290, 430)
line_end_first = (310, 360)

line_start_second = (210, 420)
line_end_second = (230, 350)

# Process each video and get the car count
car_count1 = process_video(video_path1, line_start_first, line_end_first, line_start_second, line_end_second)
car_count2 = process_video(video_path2, line_start_first, line_end_first, line_start_second, line_end_second)

# Calculate cars per minute depending on length of video
video_duration_minutes_1 = 2.75
video_duration_minutes_2 = 1.45

cars_per_minute_1 = car_count1 / video_duration_minutes_1
cars_per_minute_2 = car_count2 / video_duration_minutes_2

# Create a Pandas DataFrame to store the results
data = {
    "Video Name": [video_path1, video_path2],
    "Total Number of Cars": [car_count1, car_count2],
    "Cars Per Minute": [round(cars_per_minute_1, 0), round(cars_per_minute_2, 0)] # round up to whole number
}

df = pd.DataFrame(data)

# Display the DataFrame
print(df)

              Video Name  Total Number of Cars  Cars Per Minute
0  Traffic_Laramie_1.mp4                     6              2.0
1  Traffic_Laramie_2.mp4                     4              3.0
